In [ ]:
!pip install --upgrade azureml-sdk

In [1]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)


Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.5.0.post4 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('azureml-core~=1.3.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.5.0.post4 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('azureml-core~=1.3.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.5.0.post4 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('azureml-core~=1.3.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core.run:StepRun._fro

Azure ML SDK Version:  1.5.0


## Initialize workspace

In [2]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()

ws


Workspace.create(name='eacbmlservicews', subscription_id='f80606e5-788f-4dc3-a9ea-2eb9a7836082', resource_group='adlsgen2')

## Create an experiment

In [3]:
experiment_name = 'diabetes_dxd'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

exp


Name,Workspace,Report Page,Docs Page
diabetes_dxd,eacbmlservicews,Link to Azure Machine Learning studio,Link to Documentation


## Upload dataset

In [4]:
from azureml.core import Dataset

diabetes_dataset = Dataset.get_by_name(ws, name='diabetes')


In [5]:
diabetes = diabetes_dataset.to_pandas_dataframe().drop("Path", axis=1)

In [6]:
diabetes.head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.0,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.0,4.2905,80,135


In [7]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
AGE    442 non-null int64
SEX    442 non-null int64
BMI    442 non-null float64
BP     442 non-null float64
S1     442 non-null int64
S2     442 non-null float64
S3     442 non-null float64
S4     442 non-null float64
S5     442 non-null float64
S6     442 non-null int64
Y      442 non-null int64
dtypes: float64(6), int64(5)
memory usage: 38.1 KB


In [8]:
diabetes.describe()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
count,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,48.518100,1.468326,26.375792,94.647014,189.140271,115.439140,49.788462,4.070249,4.641411,91.260181,152.133484
std,13.109028,0.499561,4.418122,13.831283,34.608052,30.413081,12.934202,1.290450,0.522391,11.496335,77.093005
min,19.000000,1.000000,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000,25.000000
25%,38.250000,1.000000,23.200000,84.000000,164.250000,96.050000,40.250000,3.000000,4.276700,83.250000,87.000000
50%,50.000000,1.000000,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.620050,91.000000,140.500000
75%,59.000000,2.000000,29.275000,105.000000,209.750000,134.500000,57.750000,5.000000,4.997200,98.000000,211.500000
max,79.000000,2.000000,42.200000,133.000000,301.000000,242.400000,99.000000,9.090000,6.107000,124.000000,346.000000


In [9]:
features_names = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
target = "Y"

## Create a (remote) target compute

In [10]:
# Compute target creation

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "myComputeCluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print(" Cluster already exists")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           min_nodes=0, max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=30)

 Cluster already exists
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [12]:
# Retrieve existing compute target

from azureml.core.compute import ComputeTarget


compute_target_name = "myComputeCluster"
compute_target = ComputeTarget(workspace=ws, name=compute_target_name)

print(compute_target.provisioning_state)

Succeeded


## Train the model

### Train the model on a remote target compute

In [13]:
import os
script_folder = os.path.join(os.getcwd(), "remote_train")
os.makedirs(script_folder, exist_ok=True)

In [14]:
%%writefile $script_folder/train.py

import argparse
import os
import numpy as np
import glob

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
import joblib
import pickle

from azureml.core import Run
from azureml.core import Dataset
from utils import load_data

# get hold of the current run
run = Run.get_context()
exp = run.experiment
ws = run.experiment.workspace

parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg', default=0.5, help='regularization strength')
args = parser.parse_args()

# load train and test set into numpy arrays
diabetes_dataset = Dataset.get_by_name(ws, name='diabetes')
diabetes = diabetes_dataset.to_pandas_dataframe().drop("Path", axis=1)
target = "Y"
X = diabetes.drop(target, axis=1)
y = diabetes["Y"].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Train a Ridge regression model with regularization strength of', args.reg)
model = Ridge(alpha=args.reg, solver="auto", random_state=42)
model.fit(X_train, y_train)

print('Predict the test set')
y_hat = model.predict(X_test)

# calculate score on the prediction
score = model.score(X_test, y_test)
print('Score is ', score)

run.log('regularization strength', np.float(args.reg))
run.log('score', np.float(score))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_reg_remote_model.pkl')

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/computeinstance/code/Users/diabetes/remote_train/train.py


In [15]:
import shutil
shutil.copy('utils.py', script_folder)

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/computeinstance/code/Users/diabetes/remote_train/utils.py'

In [16]:
# Set up the (compute target) environnement

from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


env = Environment("diabetes_remote_env")

env.docker.enabled = True
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn',
                                                                          'pandas',
                                                                          'numpy',
                                                                          'joblib',                                                                        
                                                                          'matplotlib'
                                                                         ])
env.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")

env.python.conda_dependencies.save_to_file(".", "diabetes_env.yml")


'diabetes_env.yml'

In [17]:
from azureml.train.estimator import Estimator

script_params = {
    '--regularization': 0.5
}

estimator = Estimator(source_directory=script_folder,
              script_params=script_params,
              compute_target=cpu_cluster_name,
              environment_definition=env,
              entry_script='train.py')


In [18]:
run = exp.submit(config=estimator)
run

Experiment,Id,Type,Status,Details Page,Docs Page
diabetes_dxd,diabetes_dxd_1590994974_fc193c44,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [19]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [20]:
# specify show_output to True for a verbose log
run.wait_for_completion(show_output=True) 

RunId: diabetes_dxd_1590994974_fc193c44
Web View: https://ml.azure.com/experiments/diabetes_dxd/runs/diabetes_dxd_1590994974_fc193c44?wsid=/subscriptions/f80606e5-788f-4dc3-a9ea-2eb9a7836082/resourcegroups/adlsgen2/workspaces/eacbmlservicews

Streaming azureml-logs/20_image_build_log.txt

2020/06/01 07:03:15 Downloading source code...
2020/06/01 07:03:16 Finished downloading source code
2020/06/01 07:03:17 Creating Docker network: acb_default_network, driver: 'bridge'
2020/06/01 07:03:17 Successfully set up Docker network: acb_default_network
2020/06/01 07:03:17 Setting up Docker configuration...
2020/06/01 07:03:18 Successfully set up Docker configuration
2020/06/01 07:03:18 Logging in to registry: eacbmlservicew8493899780.azurecr.io
2020/06/01 07:03:19 Successfully logged into eacbmlservicew8493899780.azurecr.io
2020/06/01 07:03:19 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/06/01 07:03:19 Scanning for dependencies...


{'runId': 'diabetes_dxd_1590994974_fc193c44',
 'target': 'myComputeCluster',
 'status': 'Completed',
 'startTimeUtc': '2020-06-01T07:11:41.925656Z',
 'endTimeUtc': '2020-06-01T07:13:07.352926Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'e8f45a50-658c-430b-bfbe-f2a797d9e8bd',
  'azureml.git.dirty': 'True',
  'AzureML.DerivedImageName': 'azureml/azureml_a9bb5b2e7baa83080a7719e12de22936',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '640e0da3-4812-4d5d-8bbc-323e1354b451'}, 'consumptionDetails': {'type': 'Reference'}}],
 'runDefinition': {'script': 'train.py',
  'useAbsolutePath': False,
  'arguments': ['--regularization', '0.5'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'myComputeCluster',
  'dataReferences': {},
  'data': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nod

## Register the model

In [21]:
print(run.get_file_names())

['azureml-logs/20_image_build_log.txt', 'azureml-logs/55_azureml-execution-tvmps_e919f456ced86c0b0ca9bda9ffb892e234c9a2fc7b9d57a8e86a1adcdb90f1d8_d.txt', 'azureml-logs/65_job_prep-tvmps_e919f456ced86c0b0ca9bda9ffb892e234c9a2fc7b9d57a8e86a1adcdb90f1d8_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_e919f456ced86c0b0ca9bda9ffb892e234c9a2fc7b9d57a8e86a1adcdb90f1d8_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/114_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/diabetes_reg_remote_model.pkl']


In [22]:
# register model 
model = run.register_model(model_name='diabetes_ridge_regression', model_path='outputs/diabetes_reg_remote_model.pkl')

print(model.name, model.id, model.version, sep='\t')

diabetes_ridge_regression	diabetes_ridge_regression:1	1


## Create a scoring script

In [23]:
%%writefile score.py

import joblib
import numpy as np
import os

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType


# The init() method is called once, when the web service starts up.
#
# Typically you would deserialize the model file, as shown here using joblib,
# and store it in a global variable so your run() method can access it later.
def init():
    global model

    # The AZUREML_MODEL_DIR environment variable indicates
    # a directory containing the model file you registered.
    model_filename = 'diabetes_reg_remote_model.pkl'
    model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], model_filename)

    model = joblib.load(model_path)


# The run() method is called each time a request is made to the scoring API.
#
# Shown here are the optional input_schema and output_schema decorators
# from the inference-schema pip package. Using these decorators on your
# run() method parses and validates the incoming payload against
# the example input you provide here. This will also generate a Swagger
# API document for your web service.
@input_schema('data', NumpyParameterType(np.array([[59, 2, 32.1, 101.0, 157, 93.2, 38.0, 4.0, 4.8598, 87]])))
@output_schema(NumpyParameterType(np.array([151.000])))
def run(data):
    # Use the model object loaded by init().
    result = model.predict(data)

    # You can return any JSON-serializable object.
    return result.tolist()

Overwriting score.py


## Define the (inference) environement

In [24]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn'
])


## Define a inference configuration

In [25]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='score.py', environment=environment)

## Deploy in a custom environment

In [26]:
from azureml.core import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException
from azureml.core.model import Model, InferenceConfig # à ajouter ?


service_name = 'diabetes-custom-service2'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)


Running..........................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [27]:
print(service.get_logs())

2020-06-01T07:18:16,194813250+00:00 - gunicorn/run 
2020-06-01T07:18:16,195966554+00:00 - rsyslog/run 
2020-06-01T07:18:16,198822062+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_0bc8cb4f185b37c328a427a1e38cbb9b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_0bc8cb4f185b37c328a427a1e38cbb9b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_0bc8cb4f185b37c328a427a1e38cbb9b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_0bc8cb4f185b37c328a427a1e38cbb9b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_0bc8cb4f185b37c328a427a1e38cbb9b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-06-01T07:18:16,198049260+00:00 - iot-server/run 
EdgeHubC

In [28]:
print(service.scoring_uri)

http://848461b2-923e-4879-a4b9-8e956f2da3d3.westus2.azurecontainer.io/score


In [ ]:
# reload the model

from azureml.core import Webservice


service_name = 'diabetes-custom-service2'
service = Webservice(ws, service_name)

print(service.name, service.scoring_uri, sep='\t')


## Test the service

In [29]:
import json

input_payload = json.dumps({
    'data': [
        [59, 2, 32.1, 101.0, 157, 93.2, 38.0, 4.0, 4.8598, 87],
        [69, 2, 32.1, 101.0, 157, 93.2, 38.0, 4.0, 4.8598, 87]
    ],
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)


[[210.4030796084371], [211.80666178533193]]


In [ ]:
#service.delete()